In [1]:
import polars as pl
import pandas as pd
from IPython.display import display

## Load 

Load from CSV and clean up the data
1. Remove all rows where the year is 2014 because its the first year there are no pbs
2. Clean up gender as it contains "M" or "Male" and "F" or "Female". Just take the first character for simplity
3. Data Types

In [2]:
results_with_pb_df: pl.DataFrame = pl.read_csv("data/withpb_and_age-all-results.csv")


results_with_pb_df = results_with_pb_df.drop("XC_Count_of_event", "XC_Position_Min", "XC_Position_Max", "XC_Min_Age", "XC_Max_Age","exists", "from_file")

# Gender Clean up
results_with_pb_df = results_with_pb_df.with_columns(
    pl.col("Gender").str.slice(0, 1).alias("Gender")
)

results_with_pb_df = results_with_pb_df.with_columns(
    pl.col("is_pb").cast(pl.Boolean),
    pl.col("first_time_run").cast(pl.Boolean)
)

display(results_with_pb_df)




Position,Time,time_in_seconds,Bib Number,Name,Gender,Gender Position,Club,is_club_member,Points,RaceName,year,month,day,date,distance,pace,club_position,club_position_gender,is_pb,first_time_run,prior_run_attempts,prior_quickest_run_in_year,prior_quickest_run_time,prior_quickest_run_time_in_seconds,prior_quickest_run_position,prior_time_vs_current_change,prior_time_vs_current_time,prior_quickest_vs_run_time,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,wsfrl_tenure_in_runs,wsfrl_tenure_in_years,position_record_years_since_best,position_record_years_since_worst,position_record_best_year,position_record_worst_year,position_record_best_position_all_time,position_record_average_position_all_time,position_record_worst_position_all_time,p_of_10_firstname,p_of_10_surname,p_of_10_track_age_category,p_of_10_road_age_category,p_of_10_xc_age_category,p_of_10_gender,p_of_10_club,p_of_10_athlete_id,match_type,matches,parameters_used,min_age,max_age,average_age,min_dob,max_dob,avg_dob
i64,str,f64,i64,str,str,f64,str,bool,f64,str,i64,i64,i64,str,f64,str,f64,f64,bool,bool,f64,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64
3,"""00:29:14""",1754.0,880,"""Andrew Masters""","""M""",3.0,"""Lewes AC""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:39.250000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,24.0,16.0,8.0,12.0,12.0,12.0,10.0,10.0,10.0,1.0,1.0,1.0,0.0,0.0,2014.0,2014.0,12.0,12.0,12.0,"""Andrew""","""Masters""","""V50""","""V50""","""V50""","""M""","""Lewes/West 4/Wimbledon Windmil…",80881.0,"""match""",1.0,"""all""",40.0,44.0,42.0,1974.0,1970.0,1972.0
4,"""00:29:34""",1774.0,588,"""Paul Cousins""","""M""",4.0,"""Haywards Heath Harriers""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:41.750000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,9.0,5.0,4.0,2.0,5.0,7.0,10.0,10.0,10.0,3.0,3.0,1.0,0.0,0.0,2014.0,2014.0,2.0,5.0,7.0,"""Paul""","""Cousins""","""V60""","""V60""","""V60""","""M""","""Haywards Heath""",9998.0,"""match""",1.0,"""all""",50.0,54.0,52.0,1964.0,1960.0,1962.0
5,"""00:29:47""",1787.0,902,"""Michael Neville""","""M""",5.0,"""Chichester Runners""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:43.375000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,38.0,21.0,17.0,null,null,null,null,null,null,0.0,0.0,0.0,null,null,null,null,null,null,null,"""Michael""","""Neville""","""V45""","""V45""","""V45""","""M""","""Chichester""",43629.0,"""match""",1.0,"""all""",35.0,39.0,37.0,1979.0,1975.0,1977.0
6,"""00:30:07""",1807.0,918,"""Conrad Meagher""","""M""",6.0,"""Chichester Runners""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:45.875000""",2.0,2.0,false,true,0.0,null,null,null,null,null,null,null,38.0,21.0,17.0,null,null,null,null,null,null,0.0,0.0,0.0,null,null,null,null,null,null,null,"""Conrad""","""Meagher""","""SEN""","""SEN""","""SEN""","""M""","""Havant""",566746.0,"""match""",1.0,"""no_club""",13.0,24.0,18.5,2001.0,1990.0,1995.5
7,"""00:30:09""",1809.0,530,"""John Russell""","""M""",7.0,"""Portslade Hedgehoppers""",true,10.0,"""Beach Run""",2014,7,16,"""2014-07-16""",8.0,"""0:03:46.125000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,23.0,15.0,8.0,10.0,27.0,72.0,10.0,9.0,8.0,10.0,10.0,1.0,0.0,0.0,2014.0,2014.0,10.0,27.0,72.0,null,null,"""TBC""","""TBC""","""TBC""","""n""",null,null,"""missing""",0.0,"""not found""",null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
627,"""00:47:41""",2861.0,631,"""Ann Savidge""","""F""",303.0,"""Burgess 

## Do we have balanced target classes?

We calculate the PB from the prior runs. 
- The first year is 2014 which has no pbs because there is no prior data
- This needs to be removed as its unbalances the data

We also need to remove any first time only runs as a first is always a pb (technically) and will unbalance the data

In [3]:
results_with_pb_df_all = results_with_pb_df

def pivot_on_year(df: pl.DataFrame):
    df_first_timers = df.filter(pl.col("first_time_run") == True)

    number_of_first_time_runs = len(df_first_timers)
    if number_of_first_time_runs > 0:
        print(f"WARNING this data set includes {number_of_first_time_runs} first time runs that will never be a PB")
    else:
        print("PERFECT No first time runs in the data set")
    
    print(f"Total number of rows: {len(df)}")
    
    # Pivot the DataFrame
    pivot_df = df.pivot(
        values=["is_pb"],
        index="year",
        on="is_pb",
        aggregate_function="len"
    ).sort("year")
    
    display(pivot_df)



# Remove all rows where the year is 2014
results_with_pb_df_filtered = results_with_pb_df.filter(
    (pl.col("year") != 2014) & (pl.col("first_time_run") == False)
)


print ('Before')

pivot_on_year(results_with_pb_df_all)

print ('After - without 2014 and first time runs == false')

pivot_on_year(results_with_pb_df_filtered)


Before
WARNING this data set includes 33080 first time runs that will never be a PB
Total number of rows: 55523


year,false,true
i64,u32,u32
2014,6811,null
2015,6289,1116
2016,5925,1283
2017,5263,1042
2018,5269,904
2019,5451,1172
2020,378,93
2022,3967,637
2023,3758,631


After - without 2014 and first time runs == false
PERFECT No first time runs in the data set
Total number of rows: 22443


year,true,false
i64,u32,u32
2015,1116,1251
2016,1283,1821
2017,1042,2023
2018,904,2036
2019,1172,2428
2020,93,167
2022,637,1536
2023,631,1605
2024,596,2102


## Find Categoricals

These are columns with limited subset of values.

For example **education** could be 
- Graduate
- Not Graduate

In our data set
- Club 
- RaceName
- Gender
All of which are unlikely to influence a pb

However, I'll add of these are categorical variables to the model.
- RaceName
- Gender

The two `to_dummies` colum will turn these into columns with 1s and 0s.
- So for `RaceName` it will create a column for each unique value in the column and set it to 1 if the value is present and 0 if not.
- `RaceName_Downland Dash`
- `RaceName_Fittleworth`
- for each race combo





In [4]:
results_with_pb_with_dummies_df = results_with_pb_df_filtered.to_dummies(
    ["Gender", "RaceName", 'p_of_10_road_age_category','is_club_member'], drop_first=True
)


In [5]:
display(results_with_pb_with_dummies_df)

Position,Time,time_in_seconds,Bib Number,Name,Gender_F,Gender Position,Club,is_club_member_false,Points,RaceName_Downland Dash,RaceName_Fittleworth,RaceName_Great Walstead 5,RaceName_Gunpowder Trot,RaceName_Hangover-5,RaceName_Hedgehoppers 5,RaceName_Hickstead Gallop,RaceName_Highdown Hike,RaceName_Hornets Stinger,RaceName_Hove Park,RaceName_Lewes AC Easter Monday,RaceName_Run Your Heart Out 8k,RaceName_Seven Stiles,RaceName_Steepdown Challenge,RaceName_Steyning AC Roundhill Romp,RaceName_Tilgate 5,RaceName_Trundle View,RaceName_Valentine,RaceName_Windlesham-Whip,RaceName_Worthing Lido,year,month,day,date,distance,pace,club_position,…,position_record_average_position_all_time,position_record_worst_position_all_time,p_of_10_firstname,p_of_10_surname,p_of_10_track_age_category,p_of_10_road_age_category_SEN,p_of_10_road_age_category_TBC,p_of_10_road_age_category_U11,p_of_10_road_age_category_U13,p_of_10_road_age_category_U15,p_of_10_road_age_category_U17,p_of_10_road_age_category_U20,p_of_10_road_age_category_U23,p_of_10_road_age_category_V35,p_of_10_road_age_category_V40,p_of_10_road_age_category_V45,p_of_10_road_age_category_V50,p_of_10_road_age_category_V55,p_of_10_road_age_category_V65,p_of_10_road_age_category_V70,p_of_10_road_age_category_V75,p_of_10_road_age_category_V80,p_of_10_road_age_category_V85,p_of_10_road_age_category_V95,p_of_10_xc_age_category,p_of_10_gender,p_of_10_club,p_of_10_athlete_id,match_type,matches,parameters_used,min_age,max_age,average_age,min_dob,max_dob,avg_dob
i64,str,f64,i64,str,u8,f64,str,u8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i64,i64,i64,str,f64,str,f64,…,f64,f64,str,str,str,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64
7,"""00:29:11""",1751.0,167,"""Paul Cousins""",0,7.0,"""Haywards Heath Harriers""",0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:38.875000""",1.0,…,19.0,132.0,"""Paul""","""Cousins""","""V60""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""V60""","""M""","""Haywards Heath""",9998.0,"""match""",1.0,"""all""",51.0,55.0,53.0,1964.0,1960.0,1962.0
13,"""00:29:54""",1794.0,203,"""Ben Stock""",0,13.0,"""Saints and Sinners""",0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:44.250000""",2.0,…,19.0,35.0,"""Ben""","""Stock""","""V35""",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"""V35""","""M""","""Ealing Southall & Middx""",4844.0,"""many""",5.0,"""no_club""",26.0,30.0,28.0,1989.0,1985.0,1987.0
19,"""00:30:24""",1824.0,294,"""Adrian Scott""",0,19.0,"""Steyning AC""",0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:48""",1.0,…,56.0,160.0,null,null,"""TBC""",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""TBC""","""n""",null,null,"""missing""",0.0,"""not found""",null,null,null,null,null,null
21,"""00:30:29""",1829.0,390,"""Gary Witton""",0,21.0,"""Worthing Harriers""",0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:48.625000""",4.0,…,17.0,21.0,"""Gary""","""Witton""","""V50""",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"""V50""","""M""","""Worthing""",59314.0,"""match""",1.0,"""all""",41.0,45.0,43.0,1974.0,1970.0,1972.0
24,"""00:30:45""",1845.0,295,"""Scott Chinchen""",0,24.0,"""Steyning AC""",0,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2015,5,20,"""2015-05-20""",8.0,"""0:03:50.625000""",2.0,…,41.0,94.0,"""Scott""","""Chinchen""","""V35""",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"""V35""","""M""","""Steyning""",462082.0,"""match""",1.0,"""all""",26.0,30.0,28.0,1989.0,1985.0,1987.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
615,"""00:45:24""",2724.0,203,"""Verity Dods""",1,291.0,"""Horsham Joggers""",0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2016,1,31,"""2016-01-31""",6.54,"""0:06:56.513761""",38.0,…,429.0,686.0,"""Verity""","""Dods""","""V50""",0,0,0,0,0,0,0,0,0,

In [6]:
def validate_features_vs_target(df_features : pd.DataFrame, df_target : pd.DataFrame):

    if len(df_features) != len(df_target):
        print(f"WARNING Features ({len(df_features)}) != target ({len(df_target)})")
    else:
        print(f"MATCH Features_df shape: {df_features.shape} vs Target {df_target.shape}")

## Split the data

Remove the target column from the features. **is_pb**

Features are the columns that will be used to predict the target variable. 

In this case, the features are all the columns that **Known before the race starts**
- Prior performance 
- Gender
- Club mates (you would see them before the start and on course that could inspire you to run faster)
 


In [7]:


# Drop the target prediction column from the features
pb_achieved_features_df = results_with_pb_with_dummies_df.drop("is_pb")

# Info about the RACE or runner that is not useful for prediction  
#   runner
pb_achieved_features_df = pb_achieved_features_df.drop("Bib Number", "Name", "Club")
pb_achieved_features_df = pb_achieved_features_df.drop(  'p_of_10_firstname', 'p_of_10_surname', 'p_of_10_track_age_category', 'p_of_10_xc_age_category', 'p_of_10_gender', 'p_of_10_club', 'p_of_10_athlete_id')
#   race
pb_achieved_features_df = pb_achieved_features_df.drop("date", 'year', 'month', 'day', 'distance' )

# achieved time position which would be unknown at the start of the race! 
pb_achieved_features_df = pb_achieved_features_df.drop( "Time", "pace", 'Position', 'time_in_seconds','Gender Position','Points', 'club_position', 'club_position_gender', 'first_time_run' )

# Prior info on years which are versus the current year or just contains the year
pb_achieved_features_df = pb_achieved_features_df.drop( "prior_quickest_run_in_year", "prior_time_vs_current_change", "prior_time_vs_current_time", "prior_quickest_run_time", 'prior_quickest_vs_run_time', 'position_record_best_year', 'position_record_worst_year' )

pb_achieved_features_df = pb_achieved_features_df.drop(  'match_type',  'matches','parameters_used', 'min_age', 'max_age', 'average_age', 'min_dob', 'max_dob', 'avg_dob')


pb_achieved_target_df = results_with_pb_df_filtered.select("is_pb")

validate_features_vs_target(pb_achieved_features_df,pb_achieved_target_df)



display(pb_achieved_features_df)



MATCH Features_df shape: (22443, 61) vs Target (22443, 1)


Gender_F,is_club_member_false,RaceName_Downland Dash,RaceName_Fittleworth,RaceName_Great Walstead 5,RaceName_Gunpowder Trot,RaceName_Hangover-5,RaceName_Hedgehoppers 5,RaceName_Hickstead Gallop,RaceName_Highdown Hike,RaceName_Hornets Stinger,RaceName_Hove Park,RaceName_Lewes AC Easter Monday,RaceName_Run Your Heart Out 8k,RaceName_Seven Stiles,RaceName_Steepdown Challenge,RaceName_Steyning AC Roundhill Romp,RaceName_Tilgate 5,RaceName_Trundle View,RaceName_Valentine,RaceName_Windlesham-Whip,RaceName_Worthing Lido,prior_run_attempts,prior_quickest_run_time_in_seconds,prior_quickest_run_position,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,wsfrl_tenure_in_runs,wsfrl_tenure_in_years,position_record_years_since_best,position_record_years_since_worst,position_record_best_position_all_time,position_record_average_position_all_time,position_record_worst_position_all_time,p_of_10_road_age_category_SEN,p_of_10_road_age_category_TBC,p_of_10_road_age_category_U11,p_of_10_road_age_category_U13,p_of_10_road_age_category_U15,p_of_10_road_age_category_U17,p_of_10_road_age_category_U20,p_of_10_road_age_category_U23,p_of_10_road_age_category_V35,p_of_10_road_age_category_V40,p_of_10_road_age_category_V45,p_of_10_road_age_category_V50,p_of_10_road_age_category_V55,p_of_10_road_age_category_V65,p_of_10_road_age_category_V70,p_of_10_road_age_category_V75,p_of_10_road_age_category_V80,p_of_10_road_age_category_V85,p_of_10_road_age_category_V95
u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,"""1774.0""","""4.0""",13.0,8.0,5.0,2.0,20.0,132.0,10.0,9.0,7.0,9.0,10.0,2.0,1.0,0.0,2.0,19.0,132.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,"""1870.0""","""13.0""",46.0,25.0,21.0,8.0,17.0,27.0,10.0,9.0,9.0,9.0,11.0,2.0,1.0,1.0,8.0,19.0,35.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,"""1851.0""","""10.0""",15.0,10.0,5.0,10.0,59.0,160.0,10.0,8.0,5.0,7.0,8.0,2.0,1.0,1.0,10.0,56.0,160.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,"""1890.0""","""16.0""",29.0,18.0,11.0,12.0,16.0,19.0,10.0,10.0,10.0,5.0,6.0,1.0,1.0,1.0,12.0,17.0,21.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,"""1911.0""","""22.0""",15.0,10.0,5.0,22.0,38.0,94.0,10.0,8.0,7.0,9.0,11.0,2.0,1.0,1.0,22.0,41.0,94.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.0,"""2666.0""","""686.0""",37.0,14.0,23.0,285.0,469.0,686.0,1.0,1.0,1.0,7.0,19.0,2.0,2.0,1.0,226.0,429.0,686.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,"""3039.0""","""534.0""",42.0,26.0,16.0,289.0,499.0,635.0,1.0,1.0,1.0,3.0,6.0,2.0,2.0,1.0,278.0,450.0,635.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0,"""2499.0""","""657.0""",72.0,35.0,37.0,440.0,544.0,657.0,2.0,1.0,1.0,3.0,5.0,2.0,2.0,1.0,305.0,465.0,657.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

SEED = 123

(
    pb_achieved_feature_train_df,
    pb_achieved_feature_test_df,
    pb_achieved_target_train_df,
    pb_achieved_target_test_df,
) = train_test_split(
    pb_achieved_features_df,
    pb_achieved_target_df,
    test_size=0.30,
    random_state=SEED,
)

total_rows = len(pb_achieved_features_df)
train_rows = len(pb_achieved_feature_train_df)
test_rows = len(pb_achieved_feature_test_df)

test_percentage_of_total = train_rows / total_rows

print(f"Total Rows: {total_rows} Training Rows: {train_rows} Test Rows: {test_rows} Test Percentage of Total: {test_percentage_of_total:.2f}")

validate_features_vs_target(pb_achieved_feature_train_df,pb_achieved_target_train_df)
validate_features_vs_target(pb_achieved_feature_test_df,pb_achieved_target_test_df)


Total Rows: 22443 Training Rows: 15710 Test Rows: 6733 Test Percentage of Total: 0.70
MATCH Features_df shape: (15710, 61) vs Target (15710, 1)
MATCH Features_df shape: (6733, 61) vs Target (6733, 1)


## Scale the data

It is important to scale the data so that the model is not distorted by really large values.

### My notes are 
- Dimensionality reduction, which 
1. Rotates the data. Some like 45 degrees
2. Keeps the same data but reduce the complexity
- Principal Component Analysis (PCA) is a technique that can be used to reduce the dimensionality of the data.

### Notes from Copilot
The StandardScaler standardizes a feature by subtracting the mean and then scaling to unit variance. Unit variance means dividing all the values by the standard deviation.

In [9]:
scaler = StandardScaler()

pb_achieved_feature_train_scaled_df = scaler.fit_transform(
    pb_achieved_feature_train_df
)

pb_achieved_feature_test_scaled_df = scaler.fit_transform(
    pb_achieved_feature_test_df
)

print(f"Scaled Training data shape: {pb_achieved_feature_train_scaled_df.shape} vs Test {pb_achieved_feature_test_scaled_df.shape}")



Scaled Training data shape: (15710, 61) vs Test (6733, 61)


Do we have balanced target classes?

# Logistic regression example

```
from sklearn.linear_model import LogisticRegression

logistic_regression_classifier = LogisticRegression(penalty=None)

logistic_regression_classifier.fit(
    pb_achieved_feature_train_scaled_df,
    pb_achieved_target_train_df.get_column("is_pb"),
)
```

## Constraints

> ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values


One common strategy is to use the mean or median of the column to fill in the missing values. Here is how you can do it using SimpleImputer from sklearn.impute:  
Import SimpleImputer.
Create an imputer instance with the desired strategy (e.g., mean, median).
Fit the imputer on the training data and transform both the training and test data.

```
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Create an imputer instance with the mean strategy
imputer = SimpleImputer(strategy="mean")

# Fit the imputer on the training data and transform both the training and test data
pb_achieved_feature_train_scaled_df = imputer.fit_transform(pb_achieved_feature_train_scaled_df)
pb_achieved_feature_test_scaled_df = imputer.transform(pb_achieved_feature_test_scaled_df)

# Create and fit the logistic regression classifier
logistic_regression_classifier = LogisticRegression(penalty=None)

logistic_regression_classifier.fit(
    pb_achieved_feature_train_scaled_df,
    pb_achieved_target_train_df.get_column("is_pb"),
)
```





In [10]:
# Convert Polars DataFrame to Pandas DataFrame
pb_achieved_feature_train_df_pandas = pb_achieved_feature_train_df.to_pandas()
pb_achieved_target_train_df_pandas = pb_achieved_target_train_df.to_pandas()
pb_achieved_feature_test_df_pandas = pb_achieved_feature_test_df.to_pandas()
pb_achieved_target_test_df_pandas = pb_achieved_target_test_df.to_pandas()



# Define Column for Defaults

**All columns** need **default values** for **missing values**.

This can have quite an impact on the model. For example if 
- You defaulted missing **points to 10** which is the **highest point** you can score 
- You would **create an artificially high amount** of high points scores when the data is much lower

The second section of the code checks the column exist in the data frame. If they do not exist you will get an error.

> ValueError: A given column is not a column of the dataframe 

In [11]:

def get_boolean_features(df : pd.DataFrame, exclude_races = True, exclude_ages = True) -> list:
    binary_columns = []
    
    for col in df.columns:
        if exclude_races is False and col.startswith('RaceName_'):
            binary_columns.append(col)
        elif exclude_ages is False and col.startswith('p_of_10_road_age_category_'):
            binary_columns.append(col)
     
    binary_columns.append("is_club_member_false")
    binary_columns.append("Gender_F")
    
    return binary_columns


In [12]:
binary_columns = get_boolean_features(pb_achieved_feature_train_df_pandas, True, False)

print(binary_columns)

['p_of_10_road_age_category_SEN', 'p_of_10_road_age_category_TBC', 'p_of_10_road_age_category_U11', 'p_of_10_road_age_category_U13', 'p_of_10_road_age_category_U15', 'p_of_10_road_age_category_U17', 'p_of_10_road_age_category_U20', 'p_of_10_road_age_category_U23', 'p_of_10_road_age_category_V35', 'p_of_10_road_age_category_V40', 'p_of_10_road_age_category_V45', 'p_of_10_road_age_category_V50', 'p_of_10_road_age_category_V55', 'p_of_10_road_age_category_V65', 'p_of_10_road_age_category_V70', 'p_of_10_road_age_category_V75', 'p_of_10_road_age_category_V80', 'p_of_10_road_age_category_V85', 'p_of_10_road_age_category_V95', 'is_club_member_false', 'Gender_F']


In [13]:
def validate_features(features_df, 
                      low_value_good_features: list,
                      high_value_good_features: list,
                      boolean_features: list,
                      time_in_seconds_features: list,
                      years_since_value_features: list) -> list:
    
  
    feature_columns_to_validate = (low_value_good_features 
                                      + high_value_good_features 
                                      + boolean_features 
                                      + time_in_seconds_features 
                                      + years_since_value_features)
  

    # Find columns that are not in the combined feature list
    extra_columns = [col for col in features_df.columns if col not in feature_columns_to_validate]
    print(f"WARNING (Be Aware)- Column that exist in the features_df but not in the feature_columns_to_validate list: {extra_columns}")
    
    # Check if columns exist in the DataFrame
    missing_columns = [col for col in feature_columns_to_validate if col not in features_df.columns]
    missing_low_value_good_columns = [col for col in low_value_good_features if col not in features_df.columns]
    missing_high_value_good_columns = [col for col in high_value_good_features if col not in features_df.columns]
    
    good_prefix ="GOOD"
    bad_prefix ="BAD"
    
    
    # Print the results
    if missing_columns:
        print(f"{bad_prefix} - Missing columns in WHOLE set: {missing_columns}")
    else:
        print(f"{good_prefix} - All feature columns {len(feature_columns_to_validate)} exist in the DataFrame {len(features_df.columns)}.")
    
    if missing_low_value_good_columns:
        print(f"{bad_prefix} - Missing columns in low_value_good_columns: {missing_low_value_good_columns}")
    else:
        print(f"{good_prefix} - All columns in low_value_good_columns exist in the DataFrame.")
    
    if missing_high_value_good_columns:
        print(f"{bad_prefix} - Missing columns in high_value_good_columns: {missing_high_value_good_columns}")
    else:
        print(f"{good_prefix} - All columns in high_value_good_columns exist in the DataFrame.")
    
    
    # Create a DataFrame with the count of NaN values for each column
    nan_counts_df = pd.DataFrame({
        'column': features_df.columns,
        'nan_count': features_df.isna().sum()
    })
    
    # Sort the DataFrame by nan_count in descending order
    nan_counts_df = nan_counts_df.sort_values(by='nan_count', ascending=False)
    nan_counts_df = nan_counts_df[nan_counts_df['nan_count'] > 0]
    
    # Set the maximum number of rows to display
    pd.set_option('display.max_rows', 30)
    
    display(nan_counts_df.head(8))
    
    return feature_columns_to_validate

In [14]:

# Define the columns
# "Position", "Gender Position", 
time_in_seconds_columns = [ "prior_quickest_run_time_in_seconds"]

years_since_value_columns = ["position_record_years_since_best", "position_record_years_since_worst"]

low_value_good_columns = [
    "best_position_year_to_date", "average_position_year_to_date", "worst_position_year_to_date", "position_record_best_position_all_time", "position_record_worst_position_all_time", "prior_quickest_run_position",  "position_record_average_position_all_time"]


high_value_good_columns = ["fellow_club_mates_running", "fellow_club_mates_running_male", "fellow_club_mates_running_female", "average_points_year_to_date", "best_points_year_to_date", "worst_points_year_to_date", "runs_in_wsfrl_year_to_date", 'wsfrl_tenure_in_runs','wsfrl_tenure_in_years', 'prior_run_attempts']








In [15]:
# Combine the lists
all_feature_columns = validate_features(pb_achieved_feature_train_df_pandas,
                                        low_value_good_columns, 
                                        high_value_good_columns, 
                                        binary_columns, 
                                        time_in_seconds_columns, 
                                        years_since_value_columns)


WARNING (Be Aware)- Column that exist in the features_df but not in the feature_columns_to_validate list: ['RaceName_Downland Dash', 'RaceName_Fittleworth', 'RaceName_Great Walstead 5', 'RaceName_Gunpowder Trot', 'RaceName_Hangover-5', 'RaceName_Hedgehoppers 5', 'RaceName_Hickstead Gallop', 'RaceName_Highdown Hike', 'RaceName_Hornets Stinger', 'RaceName_Hove Park', 'RaceName_Lewes AC Easter Monday', 'RaceName_Run Your Heart Out 8k', 'RaceName_Seven Stiles', 'RaceName_Steepdown Challenge', 'RaceName_Steyning AC Roundhill Romp', 'RaceName_Tilgate 5', 'RaceName_Trundle View', 'RaceName_Valentine', 'RaceName_Windlesham-Whip', 'RaceName_Worthing Lido']
GOOD - All feature columns 41 exist in the DataFrame 61.
GOOD - All columns in low_value_good_columns exist in the DataFrame.
GOOD - All columns in high_value_good_columns exist in the DataFrame.


,column,nan_count
average_points_year_to_date,average_points_year_to_date,1626
best_points_year_to_date,best_points_year_to_date,1626
worst_points_year_to_date,worst_points_year_to_date,1626
worst_position_year_to_date,worst_position_year_to_date,1080
best_position_year_to_date,best_position_year_to_date,1080
average_position_year_to_date,average_position_year_to_date,1080


In [16]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Create imputers for different columns

seconds_value_imputer = SimpleImputer(strategy="constant", fill_value=400000)
low_value_imputer = SimpleImputer(strategy="constant", fill_value=400)
high_value_imputer = SimpleImputer(strategy="constant", fill_value=0) #-np.inf is too big for a float 
binary_value_imputer = SimpleImputer(strategy="constant", fill_value=np.uint8(0))


# Combine the imputers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("low_value_good", low_value_imputer, low_value_good_columns),
        ("high_value_good", high_value_imputer, high_value_good_columns),
        ("binary", binary_value_imputer, binary_columns),
        ("seconds", seconds_value_imputer, time_in_seconds_columns),
        ("years_since", SimpleImputer(strategy="constant", fill_value=0), years_since_value_columns),
    ]
)

PIPELINE_STEP_PREPROCESSOR = "preprocessor"
PIPELINE_STEP_SCALER = "scaler"

# Create a pipeline with the preprocessor, scaler, and the logistic regression classifier
pipeline = Pipeline(steps=[
    (PIPELINE_STEP_PREPROCESSOR, preprocessor),
    (PIPELINE_STEP_SCALER, StandardScaler()),
    ("classifier", LogisticRegression(penalty=None))
])

# Fit the pipeline on the training data
pipeline.fit(pb_achieved_feature_train_df_pandas, pb_achieved_target_train_df_pandas["is_pb"])

# 
# # Transform the test data
# pb_achieved_feature_test_scaled_df = pipeline.transform(pb_achieved_feature_test_df)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('low_value_good',
                                                  SimpleImputer(fill_value=400,
                                                                strategy='constant'),
                                                  ['best_position_year_to_date',
                                                   'average_position_year_to_date',
                                                   'worst_position_year_to_date',
                                                   'position_record_best_position_all_time',
                                                   'position_record_worst_position_all_time',
                                                   'prior_quickest_run_position',
                                                   'position_rec...
                                                   'is_club_member_false',
                                                   'Gender_F']),
                                                 ('seconds',
                                                  SimpleImputer(fill_value=400000,
                                                                strategy='constant'),
                                                  ['prior_quickest_run_time_in_seconds']),
                                                 ('years_since',
                                                  SimpleImputer(fill_value=0,
                                                                strategy='constant'),
                                                  ['position_record_years_since_best',
                                                   'position_record_years_since_worst'])])),
                ('scaler', StandardScaler()),
                ('classifier', LogisticRegression(penalty=None))])

In [17]:
display(binary_columns)

['p_of_10_road_age_category_SEN',
 'p_of_10_road_age_category_TBC',
 'p_of_10_road_age_category_U11',
 'p_of_10_road_age_category_U13',
 'p_of_10_road_age_category_U15',
 'p_of_10_road_age_category_U17',
 'p_of_10_road_age_category_U20',
 'p_of_10_road_age_category_U23',
 'p_of_10_road_age_category_V35',
 'p_of_10_road_age_category_V40',
 'p_of_10_road_age_category_V45',
 'p_of_10_road_age_category_V50',
 'p_of_10_road_age_category_V55',
 'p_of_10_road_age_category_V65',
 'p_of_10_road_age_category_V70',
 'p_of_10_road_age_category_V75',
 'p_of_10_road_age_category_V80',
 'p_of_10_road_age_category_V85',
 'p_of_10_road_age_category_V95',
 'is_club_member_false',
 'Gender_F']

## Run Pipeline Stages explicitly

To run the full pipeline 
`pb_achieved_feature_test_scaled_df = pipeline.transform(pb_achieved_feature_test_df)`

However this does not work 
> AttributeError: This 'Pipeline' has no attribute 'transform'

this is because `LogisticRegression` does not have a `transform` method, but a `fit`. 
  


In [19]:
from sklearn.linear_model import LogisticRegression


def run_pipeline_in_stages(df_feature : pd.DataFrame, df_target : pd.DataFrame):

    # Transform the test data using the preprocessor part of the pipeline
    preprocessed_df = pipeline.named_steps['preprocessor'].transform(df_feature)

    # Scale the preprocessed test data
    scaled_df = pipeline.named_steps['scaler'].transform(preprocessed_df)
    
    classifier = LogisticRegression(penalty=None)

    classifier.fit(
        scaled_df,
        df_target["is_pb"],
    )
    
    return scaled_df, classifier 
    

In [20]:
(scaled_train_df,
logistic_regression_classifier) = run_pipeline_in_stages(
    pb_achieved_feature_train_df_pandas,
    pb_achieved_target_train_df_pandas)





In [21]:
(scaled_test_df,
logistic_regression_classifier_junk) = run_pipeline_in_stages(
    pb_achieved_feature_test_df_pandas,
    pb_achieved_target_test_df_pandas)

In [22]:


logistic_regression_classifier.classes_



array([False,  True])

In [23]:
import altair as alt

# Convert the set to a list to maintain order
all_feature_columns_list = list(all_feature_columns)


# Create a DataFrame with the features and their coefficients
feature_coefficients_df = pl.DataFrame(
    {
        "feature": all_feature_columns_list,
        "coefficient": logistic_regression_classifier.coef_[0][:len(all_feature_columns_list)],
    }
)

# Sort the DataFrame by coefficient in descending order
sorted_feature_coefficients_df = feature_coefficients_df.sort("coefficient", descending=True)

# Plot the bar chart
alt.Chart(sorted_feature_coefficients_df.to_pandas()).mark_bar().encode(
    x="coefficient",
    y=alt.Y("feature", sort="-x")
).properties(
    width=500,
    height=300,
).display()

alt.Chart(...)

In [24]:
logistic_regression_classifier.coef_

array([[ 0.13466924, -0.13623285, -0.23337757,  0.02316406,  0.11078657,
         0.36598857, -0.30138013, -0.02178051, -0.03481172, -0.00479254,
         0.34834532,  0.01867609, -0.17151772,  0.16505345, -0.27319596,
        -0.19252722, -0.48621439,  0.07014729,  0.01313203,  0.09885819,
        -0.01269786,  0.00360294,  0.00708402, -0.00890414, -0.02272708,
         0.10201198,  0.09098333,  0.07975971,  0.03546997,  0.05243097,
         0.01806322, -0.04215785, -0.03761675, -0.06792122, -0.09045457,
        -0.1118305 ,  0.04058032, -0.07342942,  3.16879854, -0.38691731,
         0.19751714]])

In [26]:

# Make predictions on the test data
predictions = pipeline.named_steps['classifier'].predict(scaled_train_df)

# Print the predictions
print(predictions)

[False False  True ... False False False]


In [25]:
validate_features_vs_target(pb_achieved_feature_train_df_pandas, pb_achieved_target_train_df_pandas)
validate_features_vs_target(scaled_train_df, pb_achieved_target_train_df_pandas)

validate_features_vs_target(pb_achieved_feature_test_df_pandas, pb_achieved_target_test_df_pandas)

MATCH Features_df shape: (15710, 61) vs Target (15710, 1)
MATCH Features_df shape: (15710, 41) vs Target (15710, 1)
MATCH Features_df shape: (6733, 61) vs Target (6733, 1)


In [ ]:
> What are the probabilities that each of the observations in the test dataset have been assigned to the classes?

In [26]:

logistic_regression_classifier.predict_proba(scaled_test_df)

array([[0.88940263, 0.11059737],
       [0.52451895, 0.47548105],
       [0.86249264, 0.13750736],
       ...,
       [0.40690396, 0.59309604],
       [0.81886899, 0.18113101],
       [0.43119102, 0.56880898]])

In [29]:
logistic_regression_classifier.predict(scaled_test_df)

array([False, False, False, ...,  True, False,  True])



```
ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values
```

In [27]:
from sklearn.metrics import classification_report

validate_features_vs_target(scaled_test_df, pb_achieved_target_test_df)


print(
    classification_report(
        pb_achieved_target_test_df.get_column("is_pb"),
        logistic_regression_classifier.predict(scaled_test_df),
    )
)

MATCH Features_df shape: (6733, 41) vs Target (6733, 1)
              precision    recall  f1-score   support

       False       0.73      0.87      0.80      4468
        True       0.60      0.37      0.46      2265

    accuracy                           0.70      6733
   macro avg       0.66      0.62      0.63      6733
weighted avg       0.69      0.70      0.68      6733



In [28]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report

# Create an imputer instance with the mean strategy
imputer = SimpleImputer(strategy="mean")


validate_features_vs_target(pb_achieved_feature_train_scaled_df, pb_achieved_feature_test_scaled_df)

# Fit the imputer on the training data and transform both the training and test data
pb_achieved_feature_train_scaled_df = imputer.fit_transform(pb_achieved_feature_train_scaled_df)
pb_achieved_feature_test_scaled_df = imputer.transform(pb_achieved_feature_test_scaled_df)

# Fit the logistic regression classifier again
logistic_regression_classifier.fit(
    pb_achieved_feature_train_scaled_df,
    pb_achieved_target_train_df_pandas["is_pb"],
)

# Generate the classification report
print(
    classification_report(
        pb_achieved_target_test_df_pandas["is_pb"],
        logistic_regression_classifier.predict(pb_achieved_feature_test_scaled_df),
    )
)

WARNING Features (15710) != target (6733)
              precision    recall  f1-score   support

       False       0.74      0.88      0.80      4468
        True       0.62      0.38      0.47      2265

    accuracy                           0.71      6733
   macro avg       0.68      0.63      0.64      6733
weighted avg       0.70      0.71      0.69      6733



## Before we did this 

```
logistic_regression_classifier = run_pipeline_in_stages(
    pb_achieved_feature_train_df_pandas,
    pb_achieved_target_train_df_pandas)
```


In [83]:

logistic_regression_classifier_test = run_pipeline_in_stages(
    pb_achieved_feature_test_df_pandas,
    pb_achieved_target_test_df_pandas)

In [ ]:
# Generate the classification report
print(
    classification_report(
        pb_achieved_target_test_df_pandas["is_pb"],
        logistic_regression_classifier_test.predict(pb_achieved_feature_test_scaled_df),
    )
)

## Interpreting Results from the Classification Report

The classification report provides a summary of performance metrics for a classification model. Below is the detailed breakdown and interpretation:

### Definitions
1. **Precision**: The ratio of true positives to total predicted positives. Indicates how many of the predicted "True" labels are actually "True."
   \[
   \text{Precision} = \frac{\text{True Positives}}{\text{True Positives + False Positives}}
   \]

2. **Recall (Sensitivity)**: The ratio of true positives to total actual positives. Measures the ability of the classifier to identify all "True" instances.
   \[
   \text{Recall} = \frac{\text{True Positives}}{\text{True Positives + False Negatives}}
   \]

3. **F1-Score**: The harmonic mean of precision and recall, balancing their trade-off.
   \[
   \text{F1-Score} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision + Recall}}
   \]

4. **Support**: The number of actual instances in each class.

---

### Interpreting the Results

#### **Class: False (Negative Class)**
- **Precision**: `0.74`  
   - 74% of the predictions for "False" were correct.
- **Recall**: `0.88`  
   - 88% of the actual "False" cases were correctly identified by the model.
- **F1-Score**: `0.80`  
   - A strong balance between precision and recall for this class.
- **Support**: `4468`  
   - There are 4468 instances of "False" in the dataset.

#### **Class: True (Positive Class)**
- **Precision**: `0.62`  
   - 62% of the predictions for "True" were correct.
- **Recall**: `0.38`  
   - Only 38% of the actual "True" cases were correctly identified.
- **F1-Score**: `0.47`  
   - A relatively low score, indicating that the model struggles with this class.
- **Support**: `2265`  
   - There are 2265 instances of "True" in the dataset.

---

### Overall Metrics
- **Accuracy**: `0.71`  
   - The model correctly classified 71% of all instances.
- **Macro Average**:
   - **Precision**: `0.68`  
   - **Recall**: `0.63`  
   - **F1-Score**: `0.64`  
   - These averages give equal weight to both classes, regardless of support.
- **Weighted Average**:
   - **Precision**: `0.70`  
   - **Recall**: `0.71`  
   - **F1-Score**: `0.69`  
   - These averages take into account the support of each class, so the "False" class has more influence due to its larger support.

---

### Insights
1. **Class Imbalance**:  
   - There is a class imbalance, with more "False" instances (`4468`) than "True" (`2265`). This likely affects the model's ability to detect the minority class ("True").

2. **Performance on "True" Class**:
   - The model struggles to correctly identify the "True" class (low recall of `0.38`). This might indicate a need for techniques to handle class imbalance, such as oversampling, undersampling, or class weighting.

3. **Overall Performance**:
   - While the model performs reasonably well on the majority class ("False"), its performance on the minority class ("True") drags down the overall metrics.

---

### Recommendations
1. **Address Class Imbalance**:  
   - Consider using techniques like SMOTE, ADASYN, or class weighting in the logistic regression model to improve recall for the "True" class.

2. **Threshold Tuning**:  
   - Experiment with adjusting the decision threshold of the model to better balance precision and recall.

3. **Model Complexity**:  
   - Consider using a more complex model (e.g., ensemble methods like Random Forest or Gradient Boosting) if the dataset supports it.

4. **Feature Engineering**:  
   - Investigate additional features or transformations to enhance separability between the classes.

---


In [ ]:
# Assuming pb_achieved_feature_test_scaled_df is a Pandas DataFrame
# Filter out rows with NaN values
filtered_df = pb_achieved_feature_test_scaled_df.dropna()

# Display the filtered DataFrame
print(filtered_df)